### Universidad Nacional de Colombia
### Diplomado Ciencia de datos

## Análisis de componentes principales

Este caso práctico muestra un ejemplo de cómo hacer un análisis de componentes principales en R La meta del caso es conocer una de las funciones más usadas en R para hacer análisis de componentes principales. Entender las salidas que ofrece y tomar decisiones de análisis basadas en ellas.

**Contexto:** El sector de los bienes raíces es uno de los más exitosos en la economía del mundo. La gran mayoría de personas en el mundo deciden rentar sus viviendas por comodidad o economía.

**Problema de negocio:** Existen relaciones fuertes entre las variables a considerar cuando se busca un inmueble para rentarlo? Si es así cómo podría resumir esta información para lograr encontrar un lugar de renta

Empezaremos por cargar las librerías requeridas para este caso

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

## Datos
Vamos a utilizar una tabla de datos que contiene información sobre los inmuebles en renta y sus características en el mercado de Brasil. La documentación de los mismos puede encontrarse en https://www.kaggle.com/rubenssjr/brasilian-houses-to-rent?select=houses_to_rent_v2.csv Así pues, para cada inmueble se tiene la siguiente información: 
- city: ciudad donde la propiedad esta ubicada 
- area: area de la propiedad 
- rooms: número de cuartos 
- bathroom: número de baños 
- parking spaces: número de parqueaderos 
- floor: piso - animal : acepta animales 
- furnished: esta amoblada 
- hoa: administración 
- rent amount: renta 
- property tax: impuesto predial 
- fire insurance: seguro de incendio 
- total: valor total

In [ ]:
data = pd.read_csv('houses_to_rent_v2.csv')
data.head()

## Pre-procesamiento y limpieza
Debemos modificar la variable floor ya que sería interesante incluirla como un valor numérico. Sin embargo el tipo de variable es un factor y contiene el valor ‘-’ para referirse al ground floor.

In [ ]:
data['floor'].unique()

In [ ]:
data.loc[data['floor']=='-','floor'] = 0

Queremos hacer un análisis por componentes principales en datos sobre inmuebles rentados. Para acotar nuestro análisis filtraremos únicamente los registros de Sao Paulo. Adicionalmente, para ésta técnica únicamente requerimos de las variables numéricas, por lo cual excluimos del análisis las variables **animal** y **furnished**

In [ ]:
data_acp = data[data['city']=='São Paulo']
data_acp = data_acp[['area','rooms','bathroom','parking spaces','floor','hoa','rent','property_tax','fire_insurance','total']]

## Exploración
Empecemos haciendo una exploración de los datos

In [ ]:
data_acp.describe()

Podemos revisar la correlación entre las variables numéricas

In [ ]:
sns.heatmap(data_acp.corr(),annot = True)

De lo anterior ya podemos entender el funcionamiento de los datos. Aquí podemos tomar la decisión de sacar información que se vuelve redundante. En particular, la variable total resulta ser la suma de hoa, rent,property_tax y fire_insurance. Podemos observar la relación lineal casi perfecta entre el costo total y el seguro contra incendios, probablemente se calcula de acuerdo al canon de arrendamiento. En este punto podemos quedarnos únicamente con las variables hoa y rent las cuales contienen suficiente información para entender los costos.

In [ ]:
data_acp = data_acp[['area','rooms','bathroom','parking spaces','floor','hoa','rent']]

El análisis de componentes principales es sensible a las dimensiones. Es recomendable siempre hacer una estandarización de las variables a utilizar para que las unidades de análisis no interfieran en el análisis. La idea es llevar todas las variables a la misma escala, así esta no interfiere con los resultados del análisis. Esto lo podemos hacer transformando a las variables para que tengan media cero y varianza 1. Hagamos la estandarización del valor de la renta:

In [ ]:
# calculamos la media de la variable
media_renta = data_acp['rent'].mean()
# Calculamos la desviación estándar
ds_renta = data_acp['rent'].std()
# a cada valor restamos la media y dividimos por la desviación estándar
data_acp['renta_std1'] = (data_acp['rent']-media_renta)/(ds_renta)
# Revisemos el resultado
data_acp[['rent','renta_std1']].head()


In [ ]:
data_acp[['rent','renta_std1']].describe()

Existe también en Python tenemos una función **StandardScaler** que hace éste cómputo por nosotros

In [ ]:
# creamos el objeto scaler
scaler = StandardScaler()


In [ ]:
# ajustamos y transformamos los datos
data_acp['renta_std2'] = scaler.fit_transform(data_acp[['rent']]) 

In [ ]:
data_acp[['rent','renta_std1','renta_std2']].head()

In [ ]:
data_acp[['rent','renta_std1','renta_std2']].describe()

Eliminamos estas dos columnas creadas

In [ ]:
data_acp = data_acp[['area','rooms','bathroom','parking spaces','floor','hoa','rent']]

### Análisis de componentes principales

La gran mayoría de implementaciones en R o Python de ACP tienen la posibilidad de estandarizar las variables dentro de la función. En este caso utilizaremos la función PCA de la librería sklearn

In [ ]:
data_scale =  scaler.fit_transform(data_acp[['area','rooms','bathroom','parking spaces','floor','hoa','rent']]) 

In [ ]:
pca = PCA(n_components=7)

In [ ]:
pca_renta = pca.fit(data_scale)
pca_renta = pca.transform(data_scale)

Y podemos describir entonces cómo está hecha la combinación lineal de cada variable original para definir los componentes principales

In [ ]:
pca.components_

Por ejemplo, la primer componente se puede definir así:

$CP1=0.26area+0.47rooms+0.50bathroom+0.47parking−0.017floor+0.13hoa+0.44rent$

Estos pesos de cada variable en el primer componente nos dan una idea de qué información está recogiendo la dimensión.

Podemos también revisar cómo está representada cada observación en las nuevas dimensiones

In [ ]:
principalDf = pd.DataFrame(data = pca_renta, columns = ['PC1', 'PC2','PC3', 'PC4','PC5', 'PC6','PC7'])
principalDf.head()

Paso seguido podemos visualizar los valores propios para saber qué tan buena es la reducción en los componentes principales. Utilizaremos **explained_variance_** para obtener los valores propios o varianza explicada y **explained_variance_ratio_** para saber el porcentaje de varianza explicada por cada componente

In [ ]:
pca.explained_variance_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(np.array([1,2,3,4,5,6,7]),pca.explained_variance_ratio_)
ax.set(xlabel = "Dimension",
       ylabel = "Porcentaje de varianza explicada")

plt.show()


También podemos hacer una visualización de las variables originales en las nuevas dimensiones. Primero revisemos las dos primeras componentes

In [ ]:
# Get the PCA components (loadings)
PCs = pca.components_

# Use quiver to generate the basic plot
fig = plt.figure(figsize=(5,5))
plt.quiver(np.zeros(PCs.shape[1]), np.zeros(PCs.shape[1]),
           PCs[0,:], PCs[1,:], 
           angles='xy', scale_units='xy', scale=1)

# Add labels based on feature names (here just numbers)
feature_names = np.array(['area', 'rooms', 'bathroom', 'parking spaces', 'floor', 'hoa', 'rent'])

for i,j,z in zip(PCs[1,:]+0.02, PCs[0,:]+0.02, feature_names):
    plt.text(j, i, z, ha='center', va='center')

# Add unit circle
circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
plt.gca().add_artist(circle)

# Ensure correct aspect ratio and axis limits
plt.axis('equal')
plt.xlim([-1.0,1.0])
plt.ylim([-1.0,1.0])

# Label axes
plt.xlabel('PC 1')
plt.ylabel('PC 2')

# Done
plt.show()

La cercanía al circulo unitario es un indicativo de qué tan buena es ésta representación para cada variable

In [ ]:
# Get the PCA components (loadings)
PCs = pca.components_

# Use quiver to generate the basic plot
fig = plt.figure(figsize=(5,5))
plt.quiver(np.zeros(PCs.shape[1]), np.zeros(PCs.shape[1]),
           PCs[0,:], PCs[2,:], # primera y tercera dimension
           angles='xy', scale_units='xy', scale=1)

# Add labels based on feature names (here just numbers)
feature_names = np.array(['area', 'rooms', 'bathroom', 'parking spaces', 'floor', 'hoa', 'rent'])

for i,j,z in zip(PCs[2,:]+0.02, PCs[0,:]+0.02, feature_names):
    plt.text(j, i, z, ha='center', va='center')

# Add unit circle
circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
plt.gca().add_artist(circle)

# Ensure correct aspect ratio and axis limits
plt.axis('equal')
plt.xlim([-1.0,1.0])
plt.ylim([-1.0,1.0])

# Label axes
plt.xlabel('PC 1')
plt.ylabel('PC 3')

# Done
plt.show()

Podemos también visualizar los individuos en el mismo plano. Dado que tenemos 5887 observaciones la siguiente función puede tomar mucho tiempo para graficar los resultados. Por esta razón, sólo graficaremos las primeras 150 observaciones

In [ ]:
df = principalDf.loc[:150]

In [ ]:
g = sns.lmplot('PC1',
               'PC2',
               data=df,
               fit_reg=False,
               scatter=True)
               #size=7)

plt.show()

### Ejercicios

**1. ¿Qué conclusiones podríamos sacar del biplot de la primer y segunda componente?**


**2. Ajustemos el mismo análisis para la ciudad de Rio de Janeiro, ¿qué resultados se obtienen? ¿hay diferencias con el análisis anterior?**

## Conclusiones

- El análisis de componentes principales resulta ser una gran herramienta para resumir información 

- Se pueden visualizar y encontrar patrones de forma multivariada

- Es posible identificar también información recurrente